In [ ]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# 1. 데이터 로딩
df = pd.read_csv('통합된_수출_예측_데이터셋.csv', parse_dates=['날짜'])  # 데이터 통합본 넣기 // 감성지수 데이터도 포함
df.set_index('날짜', inplace=True)

# 예측 데이터셋이 xlsx 파일이면
# df = pd.read_excel('통합된_수출_예측_데이터셋.xlsx', index_col='날짜', parse_dates=True) # 데이터 통합본 넣기 // 감성지수 데이터도 포함

# 2. Lag 및 파생변수 생성
# 이전 수출량 데이터를 반영해 시계열의 연속성과 패턴을 학습하도록 lag 및 파생 변수 생성
df['lag_1'] = df['수출량'].shift(1) # 한 달 전 수출량
df['lag_2'] = df['수출량'].shift(2) # 두 달 전 수출량
df['rolling_3'] = df['수출량'].rolling(window=3).mean().shift(1) # 이전 3개월간 수출량 평균 (직전 월까지)
df['diff_1'] = df['수출량'].diff(1).shift(1) # 직전 월 대비 수출량 차이 (변동성 반영)

# 3. 결측치 제거
df = df.dropna()

# 4. 피처 & 타깃 분리
X = df.drop(columns=['수출량'])
y = df['수출량']

# 5. 시계열 순서 유지한 학습/테스트 분리
split_idx = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

# 6. LightGBM 데이터셋 변환
dtrain = lgb.dataset(data=X_train, label=y_train)
dtest = lgb.dataset(data=X_test, label=y_test)

# 7. 모델 파라미터 설정
params = {
    'max_depth': 3,
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rage' : 0.1,
    'verbose' : -1
}
num_rounds = 300

# 8. 모델 학습
model = lgb.train(params, dtrain, num_boost_round=num_rounds,
                  valid_sets=[dtrain, dtest],
                  early_stopping_rounds=30)

# 9. 예측
preds = model.predict(X_test, num_iteration=model.best_iteration)

# 10. 평가
def get_reg_eval(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")

get_reg_eval(y_test, preds)

# 11. 시각화
# 11.1 예측 값 vs 실제 값 시계열 시각화화
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.plot(y_test.index, y_test, label='실제 수출량', marker='o')
plt.plot(y_test.index, preds, label='예측 수출량', marker='x')
plt.title('실제 수출량 vs 예측 수출량 (XGBoost)')
plt.xlabel('날짜')
plt.ylabel('수출량')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 11.2 피처 중요도 시각화
xgb.plot_importance(model, importance_type='weight', height=0.5)
plt.title('XGBoost Feature Importance')
plt.tight_layout()
plt.show()